# 1.데이터 로드 및 모듈 호출

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
tada_eta = pd.read_excel('/content/drive/MyDrive/<멋쟁이사자처럼> 쏘카/Chapter_2/Chapter 2_실습파일/tada_eta.xlsx')
print(tada_eta.columns)
tada_eta.head()

Index(['ATA', 'id', 'api_eta', 'created_at_kst', 'month', 'pickup_lng',
       'pickup_lat', 'pickup_gu', 'driver_id', 'driver_lng', 'driver_lat',
       'hour'],
      dtype='object')


,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,10.60,R4L9FL6NVEL4D0,5.65,2019-12-01T00:55:09,12,127.091399,37.624545,노원구,DST52944,127.069978,37.616472,0
1,9.20,R3A7JBY7CS6TPD,8.60,2019-12-01T00:09:39,12,127.023351,37.625564,강북구,DPC72995,127.026479,37.604711,0
2,10.85,R2BIUY1TDK14XE,10.53,2019-12-15T00:18:42,12,126.881154,37.481641,금천구,DPP94923,126.906225,37.452167,0
3,5.52,R31PU5GNHSGU6C,1.92,2019-07-31T00:24:11,7,126.988375,37.567897,중구,DCT73585,126.991170,37.566416,0
4,8.60,R41WR6N1VT42DJ,7.10,2019-07-05T00:40:34,7,126.988684,37.566716,중구,DPJ86237,127.006860,37.571164,0


In [4]:
tada_eta['distance'] = ((tada_eta['pickup_lat']-tada_eta['driver_lat'])**2 + (tada_eta['pickup_lng']-tada_eta['driver_lng'])**2)*100000
tada_eta = tada_eta.drop(['id', 'created_at_kst', 'driver_id', 'pickup_lng', 'pickup_lat', 'driver_lng','driver_lat'],1)
tada_eta.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,ATA,api_eta,month,pickup_gu,hour,distance
0,10.60,5.65,12,노원구,0,52.400202
1,9.20,8.60,12,강북구,0,44.465889
2,10.85,10.53,12,금천구,0,149.731535
3,5.52,1.92,7,중구,0,1.000535
4,8.60,7.10,7,중구,0,35.014797


In [5]:
tada_eta = pd.get_dummies(tada_eta)
tada_eta.head()

,ATA,api_eta,month,hour,distance,pickup_gu_강남구,pickup_gu_강동구,pickup_gu_강북구,pickup_gu_강서구,pickup_gu_관악구,...,pickup_gu_성동구,pickup_gu_성북구,pickup_gu_송파구,pickup_gu_양천구,pickup_gu_영등포구,pickup_gu_용산구,pickup_gu_은평구,pickup_gu_종로구,pickup_gu_중구,pickup_gu_중랑구
0,10.60,5.65,12,0,52.400202,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9.20,8.60,12,0,44.465889,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.85,10.53,12,0,149.731535,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5.52,1.92,7,0,1.000535,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,8.60,7.10,7,0,35.014797,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# 2.데이터셋 분리

In [6]:
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)
train = tada_eta[:12000]
test = tada_eta[12000:]

In [7]:
x_test = test.drop('ATA',1)
y_test = test['ATA']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


# 3.이상치 제거

In [8]:
from collections import Counter 

def detect_outliers(df, n, features): 
  outlier_indices = [] 
  for col in features:
    Q1 = np.percentile(df[col], 5) 
    Q3 = np.percentile(df[col], 95) 
    IQR = Q3 - Q1 
    outlier_step = 1.5 * IQR 
    outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index 
    outlier_indices.extend(outlier_list_col) 
  outlier_indices = Counter(outlier_indices) 
  multiple_outliers = list(k for k, v in outlier_indices.items() if v > n) 

  return multiple_outliers 
  
Outliers_to_drop = detect_outliers(train, 0, ['ATA', 'api_eta', 'month', 'hour', 'distance'])

In [9]:
train.loc[Outliers_to_drop]

,ATA,api_eta,month,hour,distance,pickup_gu_강남구,pickup_gu_강동구,pickup_gu_강북구,pickup_gu_강서구,pickup_gu_관악구,...,pickup_gu_성동구,pickup_gu_성북구,pickup_gu_송파구,pickup_gu_양천구,pickup_gu_영등포구,pickup_gu_용산구,pickup_gu_은평구,pickup_gu_종로구,pickup_gu_중구,pickup_gu_중랑구
1111,41.43,8.30,12,15,16.966442,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7194,44.78,8.90,7,19,94.328891,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10449,36.93,18.45,7,15,271.284330,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,17.07,12.48,12,22,190.128496,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
81,14.72,12.20,12,15,196.772512,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10908,18.45,15.52,7,13,254.238835,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11166,35.30,4.25,12,22,1009.271325,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11741,7.95,4.93,7,5,185.393596,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11903,16.90,14.97,12,19,233.034375,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
pre_train = train.drop(Outliers_to_drop, axis=0).reset_index(drop=True)

In [11]:
x_train = pre_train.drop('ATA',1)
y_train = pre_train['ATA']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


# 4.RFECV를 통한 변수선택

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=10,random_state=42)
rfe = RFECV(model,cv=5)
rfe = rfe.fit(x_train, y_train)

print('Selected features; %s' % list(x_train.columns[rfe.support_]))

Selected features; ['api_eta', 'month', 'hour', 'distance', 'pickup_gu_강남구', 'pickup_gu_강서구', 'pickup_gu_관악구', 'pickup_gu_광진구', 'pickup_gu_구로구', 'pickup_gu_금천구', 'pickup_gu_동대문구', 'pickup_gu_동작구', 'pickup_gu_마포구', 'pickup_gu_서대문구', 'pickup_gu_서초구', 'pickup_gu_성동구', 'pickup_gu_성북구', 'pickup_gu_송파구', 'pickup_gu_양천구', 'pickup_gu_영등포구', 'pickup_gu_용산구', 'pickup_gu_은평구', 'pickup_gu_종로구', 'pickup_gu_중구']


In [ ]:
len(list(x_train.columns[rfe.support_]))

24

In [12]:
x_train_pre = x_train[['api_eta', 'month', 'hour', 'distance', 'pickup_gu_강남구', 'pickup_gu_강서구', 'pickup_gu_관악구', 'pickup_gu_광진구', 'pickup_gu_구로구', 'pickup_gu_금천구', 'pickup_gu_동대문구', 'pickup_gu_동작구', 'pickup_gu_마포구', 'pickup_gu_서대문구', 'pickup_gu_서초구', 'pickup_gu_성동구', 'pickup_gu_성북구', 'pickup_gu_송파구', 'pickup_gu_양천구', 'pickup_gu_영등포구', 'pickup_gu_용산구', 'pickup_gu_은평구', 'pickup_gu_종로구', 'pickup_gu_중구']]

# 5.OPTUNA를 활용한 하이퍼파라미터 튜닝

In [13]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 4.4 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 209 kB 46.0 MB/s 
     |████████████████████████████████| 78 kB 5.5 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
     |████████████████████████████████| 112 kB 37.8 MB/s 
     |████████████████████████████████| 146 kB 61.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=fe6f98da9ddf31db50b91ddfa8940715a518466f6d2b1090e151c6c54c934762
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [14]:
x_test_pre = x_test[['api_eta', 'month', 'hour', 'distance', 'pickup_gu_강남구', 'pickup_gu_강서구', 'pickup_gu_관악구', 'pickup_gu_광진구', 'pickup_gu_구로구', 'pickup_gu_금천구', 'pickup_gu_동대문구', 'pickup_gu_동작구', 'pickup_gu_마포구', 'pickup_gu_서대문구', 'pickup_gu_서초구', 'pickup_gu_성동구', 'pickup_gu_성북구', 'pickup_gu_송파구', 'pickup_gu_양천구', 'pickup_gu_영등포구', 'pickup_gu_용산구', 'pickup_gu_은평구', 'pickup_gu_종로구', 'pickup_gu_중구']]

In [15]:
X = np.concatenate((x_train_pre,x_test_pre), axis=0)
y = np.concatenate((y_train,y_test), axis=0)

In [21]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import PredefinedSplit
from sklearn import ensemble

In [22]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10000)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-8, 1e-2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10000)
    n_estimators =  trial.suggest_int('n_estimators', 30, 1000)
    loss = trial.suggest_categorical('loss', ['ls'])

    
    regr = ensemble.GradientBoostingRegressor(max_depth = max_depth, learning_rate = learning_rate,
                                 min_samples_leaf = min_samples_leaf,n_estimators = n_estimators,loss=loss)
    
    pds = PredefinedSplit(test_fold=[-1]*len(x_train_pre)+[0]*len(x_test_pre))
    score = cross_val_score(regr,X, y, cv=pds, scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [23]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

#Create an instance with tuned hyperparameters
optimised_Gbr = ensemble.GradientBoostingRegressor(max_depth = study.best_params['max_depth'], learning_rate = study.best_params['learning_rate'],
                                     min_samples_leaf = study.best_params['min_samples_leaf'],n_estimators = study.best_params['n_estimators'],loss = study.best_params['loss'],random_state=42)

[I 2022-06-02 21:03:04,877] A new study created in memory with name: no-name-dc0489b9-6b59-499a-8f5d-633a5e6b2c4a
[I 2022-06-02 21:03:09,537] Trial 0 finished with value: -16.835471813569807 and parameters: {'max_depth': 3509, 'learning_rate': 9.149928852835578e-08, 'min_samples_leaf': 5723, 'n_estimators': 808, 'loss': 'ls'}. Best is trial 0 with value: -16.835471813569807.
[I 2022-06-02 21:03:11,335] Trial 1 finished with value: -16.07139684848889 and parameters: {'max_depth': 3922, 'learning_rate': 0.00023457705463436068, 'min_samples_leaf': 4234, 'n_estimators': 334, 'loss': 'ls'}. Best is trial 1 with value: -16.07139684848889.
[I 2022-06-02 21:03:11,590] Trial 2 finished with value: -16.836227361308143 and parameters: {'max_depth': 9851, 'learning_rate': 8.503964969528061e-07, 'min_samples_leaf': 9263, 'n_estimators': 528, 'loss': 'ls'}. Best is trial 1 with value: -16.07139684848889.
[I 2022-06-02 21:03:14,163] Trial 3 finished with value: -16.795381761087064 and parameters: {'m

# 최종 MSE(8.2084)

In [27]:
optimised_try = ensemble.GradientBoostingRegressor(max_depth = 121, learning_rate = 0.008709674733674474,
                                     min_samples_leaf = 361,n_estimators = 960,loss = 'ls',random_state=42)

In [28]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

optimised_try.fit(x_train_pre,y_train)

mse = mean_squared_error(y_test, optimised_try.predict(x_test_pre))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
#print("The initial error of API ETA on test set: {:.4f}".format(mean_squared_error(y_test, x_test[:,0]) ))

mae = mean_absolute_error(y_test, optimised_try.predict(x_test_pre))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae))
#print("The initial error of API ETA on test set: {:.4f}".format(mean_absolute_error(y_test, x_test[:,0]) ))

The mean squared error (MSE) on test set: 8.2084
The mean absolute error (MAE) on test set: 2.0624
